In [6]:
# load entities
import json

with open(data_path+'entities.json', 'r') as fin:
    entities = json.load(fin)

corpus = []
for _id, label in entities.items():
    corpus.append({"_id": _id, "title": "", "text": label, "metadata": {}})

print(len(corpus), 'entities')
print(corpus[0])

28497 entities
{'_id': 'Q1938494', 'title': '', 'text': 'Mirosław Bork', 'metadata': {}}


In [23]:
data_path = '/ivi/ilps/personal/svakule/spoken_qa/'

path_to_questions = data_path + 'sqs_valid_wikidata2018_09_11.txt'

with open(path_to_questions) as fin:
    lines = fin.readlines()
    print(len(lines), 'questions in total')

queries, qrels = [], []
indices = []
for i, l in enumerate(lines):
    s, p, o, q = lines[i].strip('\n').split('\t')
    if s in entities:
        indices.append(i)
        queries.append({"_id": str(i), "text": q, "metadata": {}})
        qrels.append([str(i), s, '1'])

print(len(queries), 'samples with entities recognised')
print(queries[0], qrels[0])

1026 questions in total
995 samples with entities recognised
{'_id': '0', 'text': 'where was sasha vujačić born', 'metadata': {}} ['0', 'Q318926', '1']


In [16]:
# save dataset
import jsonlines

dataset = 'WD18/'

corpus_path = data_path + dataset + "entities.jsonl"
query_path = data_path + dataset + "text_questions.jsonl"
qrels_path = data_path + dataset + "dev.tsv"

with open(corpus_path, 'w') as out_file:
    for d in corpus:
        out_file.write(json.dumps(d))
        out_file.write("\n")
        
with open(query_path, 'w') as out_file:
    for d in queries:
        out_file.write(json.dumps(d))
        out_file.write("\n")

with open(qrels_path, 'w') as out_file:
    for qrel in qrels:
        out_file.write('\t'.join(qrel)+'\n')

In [17]:
from beir.datasets.data_loader import GenericDataLoader

corpus, queries, qrels = GenericDataLoader(
    corpus_file=corpus_path, 
    query_file=query_path, 
    qrels_file=qrels_path).load_custom()

# ASR Transcriptions

In [25]:
# load queries and qrels
import os
import torch
from transformers import Wav2Vec2ForCTC, Wav2Vec2Tokenizer
from datasets import load_dataset
import soundfile as sf

model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-960h")
tokenizer = Wav2Vec2Tokenizer.from_pretrained("facebook/wav2vec2-base-960h")

wav_path = "/ivi/ilps/personal/svakule/spoken_qa/sqs_valid_wikidata2018_09_11/"

queries = []
for file in os.listdir(wav_path):
    i = int(file.split('.')[0]) - 1
    if i in indices:
        speech, samplerate = sf.read(wav_path+file)

        input_values = tokenizer(speech, return_tensors="pt", padding="longest").input_values    
        logits = model(input_values).logits
        predicted_ids = torch.argmax(logits, dim=-1)
        transcription = tokenizer.batch_decode(predicted_ids)[0].lower()
        
        queries.append({"_id": str(i), "text": transcription, "metadata": {}})

print(len(queries), 'questions transcribed')
print(queries[0])

Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


995 questions transcribed
{'_id': '217', 'text': 'what is marfines van scok vix job', 'metadata': {}}


In [26]:
query_path = data_path + dataset + "wav2vec2-base-960h.jsonl"

with open(query_path, 'w') as out_file:
    for d in queries:
        out_file.write(json.dumps(d))
        out_file.write("\n")

In [27]:
from beir.datasets.data_loader import GenericDataLoader

corpus, queries, qrels = GenericDataLoader(
    corpus_file=corpus_path, 
    query_file=query_path, 
    qrels_file=qrels_path).load_custom()